# Simulator

In [1]:
import sys
sys.path.insert(0, "..")
import pandas as pd
import matplotlib.pyplot as plt
from trading_simulator import TradingSimulator, DLSimulator
from dl_solutions.dlsolutions import CryptoDLSolutions
from data_processor.data_processing import DataProcessor

Using TensorFlow backend.


## Classic

In [6]:
cryptos = ['ADA', 'BTC', 'ETH', 'LTC', 'LNK']
sel = 0

In [7]:
for crypto in cryptos:
    processor = DataProcessor([crypto])
    sim = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2, log = False)
    sim.simulate()

Loading... ADA
Total invested: 18100 €
Balance: 13172.688982503865 €
Orders won: 57
Orders lost: 122
Profit: 0.17528668411623566 

Loading... BTC
Total invested: 18000 €
Balance: 15374.282899246477 €
Orders won: 82
Orders lost: 91
Profit: 0.2985712721803598 

Loading... ETH
Total invested: 18900 €
Balance: 15762.628160203021 €
Orders won: 88
Orders lost: 96
Profit: 0.3049009608573027 

Loading... LTC
Total invested: 19400 €
Balance: 10078.209866880738 €
Orders won: 82
Orders lost: 108
Profit: 0.0040314364371514195 

Loading... LNK
Total invested: 14400 €
Balance: 13667.667598362794 €
Orders won: 70
Orders lost: 73
Profit: 0.254699138775194 



In [8]:
processor = DataProcessor([crypto])
sim1 = TradingSimulator(processor, crypto, strategy = [1], loss_allowed = 0.2)
sim2 = TradingSimulator(processor, crypto, strategy = [2], loss_allowed = 0.2)
sim3 = TradingSimulator(processor, crypto, strategy = [3], loss_allowed = 0.2)
sim4 = TradingSimulator(processor, crypto, strategy = [4], loss_allowed = 0.2)
sim5 = TradingSimulator(processor, crypto, strategy = [5], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [6], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [7], loss_allowed = 0.2)
sim7 = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2)

Loading... LNK
Loading... LNK
Loading... LNK
Loading... LNK
Loading... LNK
Loading... LNK
Loading... LNK
Loading... LNK


In [9]:
sim1.simulate()

Total invested: 200 €
Balance: 10001.092886477461 €
Orders won: 1
Orders lost: 1
Profit: 0.00546443238730717 



In [10]:
sim2.simulate()

Total invested: 1900 €
Balance: 9957.6537887469 €
Orders won: 7
Orders lost: 12
Profit: -0.022287479606894568 



In [11]:
sim3.simulate()

Total invested: 4100 €
Balance: 11848.187766539322 €
Orders won: 23
Orders lost: 18
Profit: 0.45077750403398104 



In [12]:
sim4.simulate()

Total invested: 1200 €
Balance: 10028.064948273724 €
Orders won: 5
Orders lost: 7
Profit: 0.023387456894770366 



In [13]:
sim5.simulate()

Total invested: 4200 €
Balance: 11828.971354632646 €
Orders won: 23
Orders lost: 19
Profit: 0.4354693701506299 



In [14]:
sim6.simulate()

Total invested: 100 €
Balance: 9974.066729783553 €
Orders won: 0
Orders lost: 1
Profit: -0.25933270216446547 



In [15]:
sim7.simulate()

Total invested: 14400 €
Balance: 13667.667598362794 €
Orders won: 70
Orders lost: 73
Profit: 0.254699138775194 



## DL Solution

In [3]:
crypto = 'ETH'
periods_to_retraining = 1000
prev_periods = 2
pred_periods = 20
model_selector = 'lstm'
columns = ['weekday_num', 'RSI', 'MACD', 'close']
#columns = ['mon', 'tue', 'wen', 'thu', 'fri', 'sat', 'sun', 'RSI', 'MACD', 'close']

processor = DataProcessor([crypto])
sim = DLSimulator(processor, crypto, periods_to_retraining, prev_periods, pred_periods, model_selector, columns)
#sim.simulate()
df = sim.get_df()
real_df = df.loc[139:1577]
real_df = df.loc[139:1577]
to_pred_df = df.loc[1578:1578]
real_and_pred = df.iloc[:1001]
real_df

Loading... ETH
Extracting columns columns for ETH
Proccessing and arranging columns for LSTM model


,close_1,MACD_1,RSI_1,mon_1,tue_1,thu_1,fri_1,sat_1,sun_1,close_0,MACD_0,RSI_0,mon_0,tue_0,thu_0,fri_0,sat_0,sun_0,close
139,754.99,-14.168903,51.874455,1.0,0.0,0.0,0.0,0.0,0.0,855.28,-3.037980,64.086633,0,1,0,0,0,0,993.00
140,855.28,-3.037980,64.086633,0.0,1.0,0.0,0.0,0.0,0.0,934.03,11.999501,70.431522,0,0,0,0,0,0,980.00
141,934.03,11.999501,70.431522,0.0,0.0,0.0,0.0,0.0,0.0,940.00,24.120497,70.851945,0,0,1,0,0,0,1061.00
142,940.00,24.120497,70.851945,0.0,0.0,1.0,0.0,0.0,0.0,959.30,34.881726,72.226781,0,0,0,1,0,0,1056.52
143,959.30,34.881726,72.226781,0.0,0.0,0.0,1.0,0.0,0.0,1004.11,46.489964,75.156684,0,0,0,0,1,0,1051.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573,4196.44,-9.121124,46.793026,0.0,0.0,0.0,0.0,0.0,1.0,4347.59,-5.941362,50.574650,1,0,0,0,0,0,4063.56
1574,4347.59,-5.941362,50.574650,1.0,0.0,0.0,0.0,0.0,0.0,4306.40,-6.668199,49.541304,0,1,0,0,0,0,4037.23
1575,4306.40,-6.668199,49.541304,0.0,1.0,0.0,0.0,0.0,0.0,4436.91,3.249386,52.829940,0,0,0,0,0,0,3792.75
1576,4436.91,3.249386,52.829940,0.0,0.0,0.0,0.0,0.0,0.0,4105.64,-15.443552,44.841174,0,0,1,0,0,0,3630.19


In [4]:
norm_strat = 2
strat = 1
layers = 4
batch_size = 64
epochs = 150

lstm = CryptoDLSolutions(real_df, norm_strat, strat, layers, batch_size, epochs)
lstm.build()
print(lstm.get_model().summary())
lstm.train()

Input shape: (1439, 1, 18) (1439,) (0, 1, 18) (0,)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 50)             13800     
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 50)             20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 50)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 1, 50)             20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 50)             0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dro

In [8]:
pred = lstm.predict()
print('pred:', pred)

testx (1, 1, 18)
[[[0.80736714 0.25549273 0.33147435 0.         0.         0.
   1.         0.         0.         0.84788812 0.24285511 0.39947619
   0.         0.         0.         0.         1.         0.        ]]]
test rev norm [[ 3.89794000e+03 -4.64816834e+01  4.06869161e+01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  4.08937000e+03 -5.49988324e+01  4.56817434e+01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  3.81372091e+03]]
real [3725.46]
Test RMSE: 88.261
pred: [3813.72090641]


In [5]:
to_pred_df = df.loc[1578:1578]
to_pred_df

,close_1,MACD_1,RSI_1,mon_1,tue_1,thu_1,fri_1,sat_1,sun_1,close_0,MACD_0,RSI_0,mon_0,tue_0,thu_0,fri_0,sat_0,sun_0,close
1578,3897.94,-46.481683,40.686916,0.0,0.0,0.0,1.0,0.0,0.0,4089.37,-54.998832,45.681743,0,0,0,0,1,0,3725.46


In [7]:
lstm.set_test(to_pred_df)

In [26]:
df[-20:]

,close_1,MACD_1,RSI_1,weekday_num_1,close_0,MACD_0,RSI_0,weekday_num_0,close
1559,4258.31,32.585898,47.702403,6.0,4085.97,6.845801,43.413283,0,4130.00
1560,4085.97,6.845801,43.413283,0.0,4339.44,6.820873,50.467423,1,3781.93
1561,4339.44,6.820873,50.467423,1.0,4269.36,1.133184,48.661227,2,3858.99
1562,4269.36,1.133184,48.661227,2.0,4524.85,17.045072,54.986266,3,4019.08
1563,4524.85,17.045072,54.986266,3.0,4041.20,-9.264361,43.948042,4,3958.11
1564,4041.20,-9.264361,43.948042,4.0,4095.29,-25.456716,45.271258,5,3875.42
1565,4095.29,-25.456716,45.271258,5.0,4294.78,-21.939197,49.962863,6,3959.93
1566,4294.78,-21.939197,49.962863,6.0,4444.53,-6.987417,53.205729,0,3924.14
1567,4444.53,-6.987417,53.205729,0.0,4630.26,19.622637,56.933712,1,3944.73
1568,4630.26,19.622637,56.933712,1.0,4582.95,36.473333,55.716077,2,4014.15


In [14]:
to_pred_df = pd.read_csv('/Users/gonzalozabala/Documents/Master IA/AIMaster/TFM/Code/crypto_bot/Datasets/ETH.csv', header=[1])
to_pred_df = pd.DataFrame(to_pred_df['close'].iloc[:10]).T
to_pred_df['y'] = 686
to_pred_df

,0,1,2,3,4,5,6,7,8,9,y
close,3725.46,3709.27,3630.19,3792.75,4037.23,4063.56,4094.83,4046.36,4111.41,3979.51,686


In [ ]:
processor = DataProcessor(['ETH'])
processor.load_data()
processor.clean_data('ETH')
#processor.feature_extraction('ETH')
#columns = ['close', 'Open Close Difference', 'Volume USDT' ,'Result']
#columns = ['close']
#df = processor.feature_selection('ETH', columns) 
df.loc[1588]

Loading... ETH
Extracting columns columns for ETH


close    4014.15
Name: 1588, dtype: float64

In [13]:
to_pred_df

,Result_39,Result_38,Result_37,Result_36,Result_35,Result_34,Result_33,Result_32,Result_31,Result_30,...,Result_8,Result_7,Result_6,Result_5,Result_4,Result_3,Result_2,Result_1,Result_0,Result
1163,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1,1.0


In [ ]:
'''
Estamos calculando mal la validacion pero esta funcionando
Investigar porque estamos calculando mal validacion

Cambiar test y!!!!!! en set_test
'''

In [26]:
df[1039:1059] # atentos al offset creado por null en close_0 close_1 etc

,close
1163,413.98
1164,409.11
1165,412.14
1166,405.86
1167,392.52
1168,403.45
1169,388.23
1170,387.13
1171,382.49
1172,386.46
